In [3]:
# Import necessary libraries

import requests
import json

In [4]:
# Define the STAC API endpoint
endpoint = 'https://earth-search.aws.element84.com/v0'

In [5]:
# Define the collection ID for Sentinel-2 L2A data
collection_id = 'sentinel-s2-l2a-cogs'

In [6]:
# Define the start and end date range for the data
start_date = '2021-01-01'
end_date = '2021-12-31'

In [10]:
# Load the GeoJSON file containing the area of interest
with open('/content/littoral_outline.geojson') as f:
    area_of_interest = json.load(f)


In [11]:
# Extract the geometry from the GeoJSON file
if area_of_interest['type'] == 'Feature':
    geometry = area_of_interest['geometry']
elif area_of_interest['type'] == 'FeatureCollection':
    geometry = area_of_interest['features'][0]['geometry']
else:
    geometry = area_of_interest


In [12]:
# Define the search parameters
search_params = {
    'collections': [collection_id],
    'datetime': f'{start_date}/{end_date}',
    'intersects': geometry
}


In [13]:
# Search for items using the STAC API
search_url = f'{endpoint}/search'
search_response = requests.post(search_url, json=search_params)
search_results = search_response.json()

In [22]:
# Filter for cloudless images
cloudless_items = []
for item in search_results['features']:
    # Check if the item has a 'eo:cloud_cover' property and it is less than 10%
    if 'eo:cloud_cover' in item['properties'] and item['properties']['eo:cloud_cover'] < 10:
        # Append the item to the list of cloudless items
        cloudless_items.append(item)

In [23]:
# Print the number of cloudless items found
print(f'Number of cloudless items found: {len(cloudless_items)}')

Number of cloudless items found: 2


In [25]:
# Download the cloudless images
for item in cloudless_items:
    # Get the asset URL for the RGB image
    asset_url = item['assets']['visual']['href']
    
    # Download the image
    r = requests.get(asset_url, allow_redirects=True)
    filename = f'{item["id"]}.tif'
    open(filename, 'wb').write(r.content)
    print(f'Downloaded: {filename}')

Downloaded: S2A_32NNK_20211230_0_L2A.tif
Downloaded: S2A_32NNL_20211230_0_L2A.tif
